In [1]:
import numpy as np
import sklearn
from sklearn import datasets
import matplotlib as plt
import matplotlib.pyplot as plt
import math
from sklearn.decomposition import PCA
import seaborn as sns
import plotly.graph_objects as go
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
import time

In [ ]:
#########################################################
#              PROCEDIMIENTO DE MEDIDOR                 #              
#########################################################
#   1. Separar el Dataset entre entrenamiento y test    #
#   2. Separar el Conjunto de entramiento en baseData   #
#      y poolData (Basedata puede ser 50,100...)        #
#   3. Calcular la distance entre las clases del        #
#      baseData con la muestra que vamos añadiendo      #
#   4. Calcular la distribución Softmax y la Entropía   #
#   5. Añadir el ejemplo si es de buena calidad,        #
#      podemos incluir el num de ejemplos que queramos  #
#      por ejemplo añadir solo 10                       #
#########################################################

In [2]:
def medidorDistEntr(X,y,adi,n_base):
    #---------1-----------#
    X1,X_test,y1,y_test = train_test_split(X, y,stratify=y,test_size=0.2,random_state=42,shuffle=True)
    #--------2------------#
    # Para calcular el porcentaje de samples en la base (se calcula numero*numClases)
    numClass= len(np.unique(y1)) #saber cuantas clases diferentes hay
    numTotal = X1.shape[0]
    porc = ((n_base*numClass*100)/numTotal)/100
    print(porc)
    X_Pool,X_Base,y_Pool,y_Base = train_test_split(X1, y1,stratify=y1,test_size=porc,random_state=42,shuffle=True)
    #-------3------------#
    centros=[]
    val, cont = np.unique(y_Base,return_counts=True)
    for i in val:
        ind_i=np.where(y_Base==i)
        suma = sum(X_Base[ind_i])
        centro=suma/cont[i]
        centros.append(centro)
    #-------4-------------#
    entropies = [entropy(softmax(listEuclidean(x,centros))) for x in X_Pool]
    #listEuclidean devuelve una lista con la disctancia de un ejemplo con las clases del conjunto base
    #-------5------------#
    #Primero ordeno las entropias de mayor a menos
    entroMaxs = sorted(entropies,reverse=True)[:adi] #adi se refiere al número de muestras que queremos añadir
    in_add=[]
    #Aquí busco que índices en el conjunto de datos son los que tienen la entropía mayor
    for en in entroMaxs:
        in_add.append(np.where(np.array(entropies) == en)[0][0]) #Para calcular los índices con mayor entropía
        
    X_adicionantes=X_Pool[in_add]
    y_adicionantes=y_Pool[in_add]
    #-----Finalmente añadimos al subconjunto base-----#
    X_res = np.append(X_Base,X_adicionantes,axis=0)
    y_res = np.append(y_Base,y_adicionantes)
    
    return X_res, y_res

In [3]:
def listEuclidean (p1,cen):
    return [euclidean(p1,c) for c in cen]

def euclidean (p1,p2):
    res_e = (p2-p1)**2
    suma = sum(res_e)
    return math.sqrt(suma)

def softmax(distances):
    expon = np.exp(distances)
    suma = np.sum(expon)
    return expon/suma
    
def entropy(softProb):
    op = softProb*(np.log2(softProb))
    return -1*(np.sum(op))

In [4]:
# Aqui voy a importar el data set fashion-mnist. 
import os
for dirname, _, filenames in os.walk('fashion-mnist'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
train = pd.read_csv('fashion-mnist/fashion-mnist_train.csv')
test = pd.read_csv('fashion-mnist/fashion-mnist_test.csv')

df_train = train.copy()
df_test = test.copy()

X_train = df_train.drop(['label'],axis = 1)
y_train = df_train['label']

X_test = df_test.drop(['label'],axis = 1)
y_test = df_test['label']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /=255.0

fashion-mnist\fashion-mnist_test.csv
fashion-mnist\fashion-mnist_train.csv
fashion-mnist\t10k-images-idx3-ubyte
fashion-mnist\t10k-labels-idx1-ubyte
fashion-mnist\train-images-idx3-ubyte
fashion-mnist\train-labels-idx1-ubyte


In [5]:
#Transformamos en un numpy para poder tratarlos los datos mejor
X_train=np.array(X_train)
y_train=np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [7]:
#Para la red vamos a usar una red convolucional con la siguiente estructura:
def define_model():
    model = keras.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.Dense(10, activation='softmax'))
     # compile model
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [18]:
##################################
#            Medidor             #
##################################

In [ ]:
#para una primera prueba, vamos a experimentar usando la técnica del medidor
#para ello usamos la función descrita anteriormente.
X_res,y_res = medidorDistEntr(X_train,y_train,4800,600)

In [ ]:
#Transformamos los datos en una matriz 28x28 para poder tratarlos en la red convolucional.
X_res = X_res.reshape((X_res.shape[0], 28,28 ))
X_test = X_test.reshape((X_test.shape[0],28,28))

#Como la salida de la red va a ser softmax, tenemos que pasar los datos con una codificación one hot, 
#de manera que convierte una variable categórica en un vector binario en el que solo un elemento tiene 
#el valor 1 y todos los demás elementos tienen el valor 0
y_res = to_categorical(y_res)

In [82]:
#Visualizamos las dimensiones de cada conjunto.
print(X_res.shape)
print(y_res.shape)

(10800, 28, 28)

In [84]:
model = define_model()

In [85]:
#Entrenamos el modelo mientras observamos sus métricas.
inicio=time.time()
train = model.fit(X_res, y_res, epochs=15, batch_size=32,validation_data=(X_testf, y_testf))
fin=time.time()

Epoch 1/15
338/338 [==============================] - 6s 16ms/step - loss: 0.7045 - accuracy: 0.7491 - val_loss: 0.5667 - val_accuracy: 0.7732
Epoch 2/15
338/338 [==============================] - 5s 16ms/step - loss: 0.4240 - accuracy: 0.8494 - val_loss: 0.4250 - val_accuracy: 0.8481
Epoch 3/15
338/338 [==============================] - 6s 18ms/step - loss: 0.3442 - accuracy: 0.8803 - val_loss: 0.3526 - val_accuracy: 0.8711
Epoch 4/15
338/338 [==============================] - 6s 17ms/step - loss: 0.2887 - accuracy: 0.8958 - val_loss: 0.3637 - val_accuracy: 0.8642
Epoch 5/15
338/338 [==============================] - 6s 17ms/step - loss: 0.2449 - accuracy: 0.9105 - val_loss: 0.3269 - val_accuracy: 0.8827
Epoch 6/15
338/338 [==============================] - 6s 18ms/step - loss: 0.2177 - accuracy: 0.9239 - val_loss: 0.3236 - val_accuracy: 0.8836
Epoch 7/15
338/338 [==============================] - 6s 18ms/step - loss: 0.1843 - accuracy: 0.9368 - val_loss: 0.3745 - val_accuracy: 0.8761

In [86]:
#Calculamos el tiempo que ha usado.
fin-inicio

89.27051401138306

In [ ]:
######################
#      Base          #
######################

In [35]:
#Para esta prueba vamos a analizar que ocurre cuando entrenamos la red con
#un conjunto de datos pequeño que sería el equivalante al conjunto que se ha usado anteriormente
#pero sin añadir ninguna de las muestras y transformamos de igual modo

X_train = X_train.reshape((X_train.shape[0], 28,28 ))
y_train = to_categorical(y_train)

In [ ]:
#También ara estudiar como afecta con diferentes cantidades de datos separándolas de manera estratificada 
#en esta sección del notebook se va a ir modificando el porcentaje en "test_size" de manera que se usará 
#un 10%, 25%, 30%, 40% y 50% del total de los datos.

In [272]:
# Separamos el conjunto inicial y coloco de tamaño de test un 92% para que en el entrenamiento
# use solo 6000 datos 
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.92)

In [8]:
model = define_model()

In [276]:
#entrenamos el modelo.
inicio= time.time()
train = model.fit(X_train, y_train, epochs=15, batch_size=32,validation_data=(X_test, y_test))
fin = time.time()

Epoch 1/15
150/150 [==============================] - 3s 14ms/step - loss: 0.7903 - accuracy: 0.7096 - val_loss: 0.5547 - val_accuracy: 0.7975
Epoch 2/15
150/150 [==============================] - 2s 13ms/step - loss: 0.5052 - accuracy: 0.8056 - val_loss: 0.4774 - val_accuracy: 0.8383
Epoch 3/15
150/150 [==============================] - 2s 13ms/step - loss: 0.4304 - accuracy: 0.8429 - val_loss: 0.4538 - val_accuracy: 0.8367
Epoch 4/15
150/150 [==============================] - 2s 13ms/step - loss: 0.3761 - accuracy: 0.8629 - val_loss: 0.4130 - val_accuracy: 0.8617
Epoch 5/15
150/150 [==============================] - 2s 13ms/step - loss: 0.3130 - accuracy: 0.8848 - val_loss: 0.3981 - val_accuracy: 0.8625
Epoch 6/15
150/150 [==============================] - 2s 13ms/step - loss: 0.2853 - accuracy: 0.8990 - val_loss: 0.3880 - val_accuracy: 0.8625
Epoch 7/15
150/150 [==============================] - 2s 15ms/step - loss: 0.2462 - accuracy: 0.9087 - val_loss: 0.3428 - val_accuracy: 0.8842

In [277]:
#calculamos el tiempo.
fin-inicio

31.705026626586914

In [250]:
####################
#     COMPLETO     #
####################

In [9]:
#Finalmente voy a estudiar el rendimiento del modelo usando todos 
#los datos que nos proporciona el dataset.

#Para ello transformamos los datos como en las anteriores ocasiones
X_train = X_train.reshape((X_train.shape[0], 28,28 ))
y_train = to_categorical(y_train)

In [65]:
#Separamos para tener conjuto de test
X_train_com, X_test_com, y_train_com, y_test_com = train_test_split(X_train, y_train, stratify=y_train, test_size = 0.20)

In [19]:
y_test = to_categorical(y_test)

In [25]:
X_test = X_test.reshape((X_test.shape[0], 28,28 ))

In [71]:
model = define_model()

In [74]:
#entrenamos
inicio=time.time()
train = model.fit(X_train, y_train, epochs=15, batch_size=64,validation_data=(X_test, y_test))
fin=time.time()

Epoch 1/15
938/938 [==============================] - 20s 21ms/step - loss: 0.0390 - accuracy: 0.9878 - val_loss: 0.3030 - val_accuracy: 0.9217
Epoch 2/15
938/938 [==============================] - 21s 22ms/step - loss: 0.0314 - accuracy: 0.9908 - val_loss: 0.3186 - val_accuracy: 0.9221
Epoch 3/15
938/938 [==============================] - 21s 22ms/step - loss: 0.0265 - accuracy: 0.9927 - val_loss: 0.3272 - val_accuracy: 0.9183
Epoch 4/15
938/938 [==============================] - 22s 23ms/step - loss: 0.0229 - accuracy: 0.9940 - val_loss: 0.3515 - val_accuracy: 0.9219
Epoch 5/15
938/938 [==============================] - 21s 23ms/step - loss: 0.0201 - accuracy: 0.9949 - val_loss: 0.3507 - val_accuracy: 0.9216
Epoch 6/15
938/938 [==============================] - 22s 23ms/step - loss: 0.0187 - accuracy: 0.9953 - val_loss: 0.3473 - val_accuracy: 0.9191
Epoch 7/15
938/938 [==============================] - 21s 22ms/step - loss: 0.0155 - accuracy: 0.9964 - val_loss: 0.3613 - val_accuracy:

In [73]:
fin-inicio

405.6341555118561

In [ ]:
##################################################################################
#                                        CNN                                     #
##################################################################################
#  porcentaje  #    10%    #    25%    #    30%    #   40%   #   50%   #   80%   #
##################################################################################
# Num muestras #    6000    #  15000   #   18000   #  24000  #  30000  #  48000  #
# Precisión    #    0.87    #   0.89   #    0.90   #   0.90  #  0.90   #  0.92   #
# Tiempo       #    57.01   #  112.87  #  137.33   # 165.90  #  205.81 # 318.88  #
##################################################################################

In [294]:
######################################################
#                         CNN                        #
######################################################
#  Conjunto   #    prec    #   tiempo    #  Muestras #
######################################################
# Indicador   #    0.88    #   36.9167   # 6000 (5000 + 1000)   #
# Estratifi   #    0.88    #   31.705    #    6000   #
# Completo    #    0.98    #   388.753   #   60000   #
######################################################

In [7]:
val, cont = np.unique(y_train,return_counts=True)

In [94]:
####################################################
#                     Random                       #
####################################################

In [ ]:
#Para finalizar vamos a estudiar que pasaría al separar los datos de manera aleatoria y no estratificada.
# Voy a randomizar el dataset y separarlo en test y train una vez aleatorizado.

In [180]:
randomize = np.arange(len(y_train))

In [181]:
np.random.shuffle(randomize)

In [183]:
X_train = X_train.reshape((X_train.shape[0], 28,28 ))
y_train = to_categorical(y_train)

In [184]:
X_trains = X_train[randomize]
y_trains = y_train[randomize]

In [186]:
porc=0.2*len(X_trains)

In [188]:
perc = int(porc)

In [189]:
X_trainRand = X_trains[0:perc]
X_testRand = X_trains[perc:]
Y_trainRand = y_trains[0:perc]
Y_testRand = y_trains[perc:]

In [190]:
len(Y_trainRand)

12000

In [119]:
val, cont = np.unique(Y_trainRand,return_counts=True)

In [191]:
model = define_model()

In [192]:
inicio=time.time()

In [193]:
inicio=time.time()
train = model.fit(X_trainRand, Y_trainRand, epochs=15, batch_size=32,validation_data=(X_testRand, Y_testRand))
fin=time.time()

Epoch 1/15
375/375 [==============================] - 11s 28ms/step - loss: 0.6102 - accuracy: 0.7844 - val_loss: 0.5099 - val_accuracy: 0.8145
Epoch 2/15
375/375 [==============================] - 11s 30ms/step - loss: 0.3877 - accuracy: 0.8652 - val_loss: 0.4133 - val_accuracy: 0.8561
Epoch 3/15
375/375 [==============================] - 12s 31ms/step - loss: 0.3313 - accuracy: 0.8815 - val_loss: 0.3667 - val_accuracy: 0.8722
Epoch 4/15
375/375 [==============================] - 12s 33ms/step - loss: 0.2879 - accuracy: 0.8977 - val_loss: 0.3390 - val_accuracy: 0.8821
Epoch 5/15
375/375 [==============================] - 12s 33ms/step - loss: 0.2474 - accuracy: 0.9098 - val_loss: 0.3720 - val_accuracy: 0.8726
Epoch 6/15
375/375 [==============================] - 12s 31ms/step - loss: 0.2210 - accuracy: 0.9191 - val_loss: 0.3298 - val_accuracy: 0.8847
Epoch 7/15
375/375 [==============================] - 12s 31ms/step - loss: 0.1986 - accuracy: 0.9292 - val_loss: 0.3415 - val_accuracy:

In [194]:
fin-inicio


176.87053394317627

In [ ]:
######################################################
#                         CNN                        #
######################################################
#  Conjunto   #    prec    #   tiempo    #  Muestras #
######################################################
# Indicador   #    0.88    #   36.9167   # 6000 (5000 + 1000)   #
# Estratifi   #    0.88    #   31.705    #    5000   #
# Completo    #    0.98    #   388.753   #   60000   #
#  Random     #    0.86    #   55.208    #   5000   #
######################################################